In [4]:
from pinecone import Pinecone
import pickle
from rag.pinecone_query import PineconeRetriever
from rag.pipeline import RAG_Piepline
from config import PINECONE_API_KEY, GROQ_API_KEY, LANGCHAIN_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "textbook-rag"
mapping_path = "data/chunks/mapping.pickle"
embedding_mdl = "multilingual-e5-large"
k = 3

with open(mapping_path, 'rb') as f:
    mapping = pickle.load(f)

retriever = PineconeRetriever(pc=pc, index_name=index_name, embedding_mdl=embedding_mdl, k=k, mapping=mapping)
rag = RAG_Piepline(retriever, GROQ_API_KEY, LANGCHAIN_API_KEY)

reply = rag.get_response("List some uses of ultrasound")
print(reply)

Some uses of ultrasound are:

1. Cleaning parts located in hard-to-reach places.
2. Detecting cracks and flaws in metal blocks.
3. Echocardiography (forming the image of the heart).
4. Getting images of internal organs of the human body (using an ultrasound scanner).
5. Cleaning objects such as spiral tubes, odd shaped parts, and electronic components.
